# Financial Statements in the OpenBB Platform

OpenBB Platform data extensions provide access to financial statements as quarterly or annual.  There are also endpoints for ratios and other common non-GAAP metrics.  Most data providers require a subscription to access all data. Refer to the website of a specific provider for details on entitlements and coverage.

Financial statement functions are grouped under the `obb.equity.fundamental` module.

## Endpoints

The typical financial statements consist of three endpoints:

- Balance Sheet: `obb.equity.fundamental.balance()`
- Income Statement: `obb.equity.fundamental.income()`
- Cash Flow Statement: `obb.equity.fundamental.cash()`

The main parameters are:

- `symbol`: The company's symbol.
- `period`: 'annual' or 'quarter'. Default is 'annual'.
- `limit`: Limit the number of results returned, from the latest. Default is 5. For perspective, 150 will go back to 1985. The amount of historical records varies by provider.

### Field Names

Some considerations to keep in mind when working with financial statements data are:

- Every data provider has their own way of parsing and organizing the three financial statements.
- Items within each statement will vary by source and by the type of company reporting.
- Names of line items will vary by source.
- "Date" values may differ because they are from the period starting/ending or date of reporting.

This example highlights how different providers will have different labels for compnay facts.

In [1]:
from openbb import obb
import pandas as pd

df = pd.DataFrame()

df["yfinance"] = (
  obb.equity.fundamental.balance("TGT", provider="yfinance", limit=4)
  .to_df()["TotalAssets"].reset_index(drop=True)
)

df["fmp"] = (
  obb.equity.fundamental.balance("TGT", provider="fmp", limit=4)
  .to_df()["assets"].convert_dtypes().reset_index(drop=True)
)

df["intrinio"] = (
  obb.equity.fundamental.balance("TGT", provider="intrinio", limit=4)
  .to_df()["total assets"].convert_dtypes().reset_index(drop=True)
)

df["polygon"] = (
  obb.equity.fundamental.balance("TGT", provider="polygon", limit=4)
  .to_df()["total_assets"].convert_dtypes().reset_index(drop=True)
)

df


,yfinance,fmp,intrinio,polygon
0,42779000000,42779000000,42779000000,42779000000
1,51248000000,51248000000,51248000000,51248000000
2,53811000000,53811000000,53811000000,53811000000
3,53335000000,53335000000,53335000000,53335000000


### Weighted Average Shares Outstanding

This key metric will be found under the income statement.  It might also be called, 'basic', and the numbers do not include authorized but unissued shares.  A declining count over time is a sign that the company is returning capital to shareholders in the form of buy backs.  Under ideal circumstances, it is more capital-efficient, for both company and shareholders, because distributions are double-taxed.  The company pays income tax on paid dividends, and the beneficiary pays income tax again on receipt.

A company will disclose how many shares are outstanding at the end of the period  as a weighted average over the reporting period - three months.

Let's take a look at Target.  To make the numbers easier to read, we'll divide the entire column by one million.

In [2]:
data = (
  obb.equity.fundamental.income("TGT", provider="fmp", limit=150, period="quarter")
  .to_df()
)

shares = data["weighted_average_shares_outstanding"]/1000000

display(shares.head(1))

display(shares.tail(1))


date
1986-07-31    1168.816
Name: weighted_average_shares_outstanding, dtype: float64

date
2023-10-31    461.6
Name: weighted_average_shares_outstanding, dtype: float64

Thirty-seven years later, the share count is approaching a two-thirds reduction.  12.2% over the past five years.  In four reporting periods, 1.3 million shares have been taken out of the float.

In [3]:
display(shares.pct_change(20).iloc[-1])

display(shares.iloc[-4] - shares.iloc[-1])


-0.12226659060657907

-1.3000000000000114

With an average closing price of $143.37, that represents approximately $190M in buy backs.

In [4]:
price = (
  obb.equity.price.historical("TGT", start_date="2022-10-01", provider="fmp")
  .to_df()
)

round((price["close"].mean()*1300000)/1000000, 2)


187.55

### Dividends Paid

Dividends paid is in the cash flow statement.  We can calculate the amount-per-share with the reported data.

In [5]:
dividends = (
  obb.equity.fundamental.cash("TGT", provider="fmp", limit=150, period="quarter")
  .to_df()[["dividends_paid"]]
)

dividends["shares"] = data["weighted_average_shares_outstanding"]
dividends["div_per_share"] = dividends["dividends_paid"]/dividends["shares"]

dividends["div_per_share"].tail(4)


date
2023-01-28   -1.079731
2023-04-29   -1.078325
2023-07-29   -1.081023
2023-10-31   -1.098354
Name: div_per_share, dtype: float64

This can be compared against the real amounts paid to common share holders, as announced.  Note that the dates above represent the report date, and that dividends paid are attributed to the quarter they were paid in.  The value from "2023-01-28" equates to the fourth quarter of 2022.

In [6]:
(
  obb.equity.fundamental.dividends("TGT", provider="fmp")
  .to_df()["dividend"]
  .loc["2022-11-15":"2023-08-15"]
)


date
2022-11-15    1.08
2023-02-14    1.08
2023-05-16    1.08
2023-08-15    1.10
Name: dividend, dtype: float64

The numbers check out, and the $2B paid to investors over four quarters is more than ten times the $190M returned through share buy backs.

### Financial Attributes

The `openbb-intrinio` data extension has an endpoint for extracting a single fact from financial statements.  There is a helper function for looking up the correct `tag`.

#### Search Financial Attributes

Search attributes by keyword.

In [7]:
(
    obb.equity.fundamental.search_attributes("marketcap")
    .to_df()
)


,id,name,tag,statement_code,statement_type,parent_name,sequence,factor,transaction,type,unit
0,tag_BgkbWy,Market Capitalization,marketcap,calculations,industrial,None,NaN,None,None,valuation,usd
1,tag_kylOqz,Market Capitalization,marketcap,calculations,financial,None,NaN,None,None,valuation,usd
2,tag_XLRlqy,Market Sector,market_sector,current,None,None,NaN,None,None,security,string
3,tag_2gBA8y,Market Category,market_category,current,None,None,NaN,None,None,security,string
4,tag_DzonXe,Marketing Expense,marketingexpense,income_statement,industrial,totaloperatingexpenses,9.0,+,debit,income_statement_metric,usd
...,...,...,...,...,...,...,...,...,...,...,...
95,tag_nzJAmX,Total Long-Term Debt,ltdebtandcapleases,calculations,financial,None,NaN,None,None,metric,usd
96,tag_9XaL5g,Other Net Changes in Cash,othernetchangesincash,cash_flow_statement,industrial,netchangeincash,33.0,+,debit,cash_flow_statement_metric,usd
97,tag_5X7p6z,Other Net Changes in Cash,othernetchangesincash,cash_flow_statement,financial,netchangeincash,37.0,+,debit,cash_flow_statement_metric,usd
98,tag_qzEwng,"Changes in Operating Assets and Liabilities, net",increasedecreaseinoperatingcapital,cash_flow_statement,financial,netcashfromcontinuingoperatingactivities,8.0,+,debit,cash_flow_statement_metric,usd


The `tag` is what we need, in this case it is what we searched for.

In [8]:
marketcap = (
  obb.equity.fundamental.historical_attributes(symbol="TGT", tag = "marketcap", period="quarter")
  .to_df()
)

marketcap.tail(5)


,value
date,
2022-12-31,6.692963e+10
2023-03-31,7.502370e+10
2023-06-30,5.991695e+10
2023-09-30,5.061437e+10
2023-11-22,6.049500e+10


Doing some quick math, and ignoring the most recent value, we can see that the market cap of Target was down nearly a quarter over the last four reporting periods.

In [9]:
(
    (marketcap.loc["2023-09-30"] - marketcap.loc["2022-12-31"])/marketcap.loc["2022-12-31"]
).value


-0.24376733082703084

## Ratios and Other Metrics

Other valuation functions are derivatives of the financial statements, but the data provider does the math.  Values are typically ratios between line items, on a per-share basis, or as a percent growth.

This data set is where you can find EPS, FCF, P/B, EBIT, quick ratio, etc.

### Quick Ratio

Target's quick ratio could be one reason why its share price is losing traction against the market.  Its ability to pay current obligations is not optimistically reflected in a 0.27 score, approximately 50% below the historical median.

In [28]:
ratios = (
  obb.equity.fundamental.ratios("TGT", limit=50, provider="fmp")
  .to_df()
)

display(f"Current Quick Ratio: {round(ratios['quick_ratio'].iloc[-1], 4)}")
display(f"Median Quick Ratio: {round(ratios['quick_ratio'].median(), 4)}")


'Current Quick Ratio: 0.2701'

'Median Quick Ratio: 0.5844'

### Free Cash Flow Yield

The `metrics` endpoint, with the `openbb-fmp` data extension, has a field for free cash flow yield.  It is calculated by taking the free cash flow per share divided by the current share price.  We could arrive at this answer by writing some code, but these types of endpoints do the work so we don't have to.  This is part of the value-add that API data distributors provide, they allow you to get straight to work with data.

We'll use this endpoint to extract the data, and compare with some of Target's competition over the last ten years.

In [26]:
# List of other retail chains
tickers = ["COST", "BJ", "DLTR", "DG", "WMT", "BIG", "M", "KSS", "TJX"]

# Create a dictionary of tickers and company names.
names = {
    ticker: obb.equity.fundamental.overview(ticker, provider="fmp").results.company_name
    for ticker in tickers
}
# Create a column for each.
fcf_yield = pd.DataFrame()
for ticker in tickers:
    fcf_yield[names[ticker]] = (
        obb.equity.fundamental.metrics(ticker, provider="fmp", period="annual", limit=10)
        .to_df()
        .reset_index()
        .set_index("calendar_year")
        .sort_index(ascending=False)
        ["free_cash_flow_yield"]
    )
fcf_yield.transpose()


calendar_year,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014
Costco Wholesale Corporation,0.027922,0.014860,0.026582,0.039351,0.025906,0.027438,0.060884,0.008941,0.030741,0.037483
"BJ's Wholesale Club Holdings, Inc.",NaN,0.044709,0.067213,0.113551,0.056630,0.091107,0.026186,0.065871,0.016947,NaN
"Dollar Tree, Inc.",NaN,0.010756,0.013957,0.075627,0.040338,0.041252,0.034069,0.063465,0.016602,0.041047
Dollar General Corporation,NaN,0.008256,0.037507,0.058973,0.036922,0.046197,0.042609,0.050776,0.039524,0.046052
Walmart Inc.,0.031243,0.028372,0.065462,0.044591,0.062023,0.057275,0.101038,0.073506,0.059712,0.041544
"Big Lots, Inc.",NaN,-0.550469,0.025262,0.115757,0.069464,-0.111853,0.037219,0.100721,0.110443,0.089253
"Macy's, Inc.",NaN,0.050473,0.270980,0.039111,0.091301,0.101426,0.155761,0.098993,0.065634,0.072322
Kohl's Corporation,NaN,-0.143961,0.189677,0.147968,0.119492,0.139799,0.096137,0.198790,0.081652,0.110697
"The TJX Companies, Inc.",0.027159,0.023497,0.051769,0.040167,0.048827,0.039935,0.053697,0.043328,0.046442,0.040643


There are more usage examples on our [website](https://docs.openbb.co/platform/usage)